# 2D Injector Optimization with NN Prior at LCLS
Aiming to optimize transverse beam size in 2D

In [ ]:
import os
import socket

hostname = socket.gethostname()
if hostname == "lcls-srv04":
    os.environ["OMP_NUM_THREADS"] = str(1)
elif hostname == "test-rhel7":
    os.environ["OMP_NUM_THREADS"] = str(6)

In [ ]:
# optionally add scripts location to path
if True:
    import sys
    sys.path.append("../../")
    sys.path.append("../")

run_dir = "/home/physics3/ml_tuning/20231120_LCLS_Injector/"
if not os.path.exists(run_dir):
    os.makedirs(run_dir)
print(sys.path)

## Set up image diagnostic

In [ ]:
from scripts.image import ImageDiagnostic
import yaml

fname = "../OTR3_config.yml"
image_diagnostic = ImageDiagnostic.parse_obj(yaml.safe_load(open(fname)))
image_diagnostic.save_image_location = run_dir
image_diagnostic.n_fitting_restarts = 2
image_diagnostic.visualize = False
image_diagnostic.background_file = run_dir + "OTRS_IN20_621_background.npy"
print(image_diagnostic.yaml())

In [ ]:
#image_diagnostic.measure_background(file_location=run_dir)

In [ ]:
image_diagnostic.background_file

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(image_diagnostic.background_image)

In [ ]:
image_diagnostic.test_measurement()

## Define VOCS

In [ ]:
import pandas as pd
from xopt import VOCS

In [ ]:
filename = "../variables.csv"
VARIABLE_RANGES = pd.read_csv(filename, index_col=0, header=None).T.to_dict(orient='list')

IMAGE_CONSTRAINTS = {
    "bb_penalty": ["LESS_THAN", 0.0],
    "log10_total_intensity": ["GREATER_THAN", image_diagnostic.min_log_intensity]
}

In [ ]:
VARIABLES = ["SOLN:IN20:121:BCTRL", "QUAD:IN20:121:BCTRL"]

vocs = VOCS(
    variables = {ele: VARIABLE_RANGES[ele] for ele in VARIABLES},
    constraints = IMAGE_CONSTRAINTS,
    objectives = {"total_size": "MINIMIZE"},
)
print(vocs.as_yaml())

## Define evaluate function

In [ ]:
from time import sleep

import torch
import numpy as np
from epics import caput, caget_many, caget

from utils import get_model_predictions, numpy_save

In [ ]:
objective_scale = 1e-3

In [ ]:
def eval_beamsize(input_dict, generator = None):
    global image_diagnostic
    # set PVs
    for k, v in input_dict.items():
        print(f'CAPUT {k} {v}')
        caput(k, v)

    sleep(5.0)

    # get beam sizes from image diagnostic
    metadata = input_dict
    results = image_diagnostic.measure_beamsize(1, **metadata)
    results["S_x_mm"] = np.array(results["Sx"]) * 1e-3
    results["S_y_mm"] = np.array(results["Sy"]) * 1e-3

    # get other PV's NOTE: Measurements not synchronous with beamsize measurements!
    results = results

    # add total beam size
    sigma_xy = np.sqrt(np.array(results["Sx"]) ** 2 + np.array(results["Sy"]) ** 2)
    roundness = np.abs(np.array(results["Sx"]) - np.array(results["Sy"]))
    results["sigma_xy"] = sigma_xy
    results["total_size"] = objective_scale * (sigma_xy + roundness)    
    # results["total_size"] = np.sqrt(np.abs(np.array(results["Sx"])) * np.array(results["Sy"]))
    
    # GP model predictions
    model_predictions = get_model_predictions(input_dict, generator)
    results.update(model_predictions)

    numpy_save()
    
    return results

## Define NN prior

In [ ]:
from lume_model.utils import variables_from_yaml
from lume_model.models import TorchModel, TorchModule

sys.path.append("calibration/calibration_modules/")
from decoupled_linear import OutputOffset, OutputScale, DecoupledLinearOutput

In [ ]:
model_path = "lcls_cu_injector_nn_model/"

# # load nn_to_cal transformers
# reg = "low"  # "low" or "high"
# input_nn_to_cal = torch.load(f"calibration/input_nn_to_cal_{reg}_reg.pt")
# output_nn_to_cal = torch.load(f"calibration/output_nn_to_cal_{reg}_reg.pt")

# load sim_to_nn transformers
input_sim_to_nn = torch.load(model_path + "model/input_sim_to_nn.pt")
output_sim_to_nn = torch.load(model_path + "model/output_sim_to_nn.pt")

# load pv_to_sim transformers
input_pv_to_sim = torch.load(model_path + "model/input_pv_to_sim.pt")
output_pv_to_sim = torch.load(model_path + "model/output_pv_to_sim.pt")

# load in- and output variable specification
input_variables, output_variables = variables_from_yaml(model_path + "model/pv_variables.yml")
# input_variables, output_variables = variables_from_yaml(f"calibration/pv_variables_{reg}_reg_pydantic2.yml")

# replace keys in input variables
for var in input_variables:
    var.name = var.name.replace("BACT", "BCTRL")

# create TorchModel
lume_model = TorchModel(
    model=model_path + "model/model.pt",
    input_variables=input_variables,
    output_variables=output_variables,
    input_transformers=[input_pv_to_sim, input_sim_to_nn],
    output_transformers=[output_sim_to_nn, output_pv_to_sim],
    # input_transformers=[input_pv_to_sim, input_sim_to_nn, input_nn_to_cal],
    # output_transformers=[output_nn_to_cal, output_sim_to_nn, output_pv_to_sim],
)

# wrap in TorchModule
lume_module = TorchModule(
    model=lume_model,
    input_order=vocs.variable_names,
    output_order=lume_model.output_names[0:2],
)

# define objective model
class ObjectiveModel(torch.nn.Module):
    def __init__(self, model: TorchModule):
        super(ObjectiveModel, self).__init__()
        self.model = model

    @staticmethod
    def function(sigma_x: torch.Tensor, sigma_y: torch.Tensor) -> torch.Tensor:
        # using this calculation due to occasional negative values
        sigma_xy = torch.sqrt(sigma_x ** 2 + sigma_y ** 2)
        roundness = torch.abs(sigma_x - sigma_y)
        return (sigma_xy + roundness) * objective_scale

    def forward(self, x) -> torch.Tensor:
        idx_sigma_x = self.model.output_order.index("OTRS:IN20:571:XRMS")
        idx_sigma_y = self.model.output_order.index("OTRS:IN20:571:YRMS")
        sigma_x = self.model(x)[..., idx_sigma_x]
        sigma_y = self.model(x)[..., idx_sigma_y]
        return self.function(sigma_x, sigma_y)


objective_model = ObjectiveModel(lume_module)

In [ ]:
# define custom mean
objective_model.requires_grad_(False);

custom_mean = objective_model
# custom_mean = OutputOffset(
#     model=objective_model,
# )

## Restrict ranges based on profile monitor

In [ ]:
vocs.variables = {k: input_variables[lume_model.input_names.index(k)].value_range for k in vocs.variable_names}
vocs.variables["SOLN:IN20:121:BCTRL"] = [0.467, 0.479]
print(vocs.as_yaml())

## Run Xopt

In [ ]:
from xopt import Xopt, VOCS
from xopt.evaluator import Evaluator
from xopt.numerical_optimizer import LBFGSOptimizer
from xopt.generators.bayesian import ExpectedImprovementGenerator
from xopt.generators.bayesian.models.standard import StandardModelConstructor

# remember to set use low noise prior to false!!!
gp_constructor = StandardModelConstructor(
    use_low_noise_prior=False,
    mean_modules={"total_size": custom_mean},
    trainable_mean_keys=["total_size"],
)
generator = ExpectedImprovementGenerator(
    vocs=vocs,
    gp_constructor=gp_constructor,
)
generator.numerical_optimizer.max_iter = 200
evaluator = Evaluator(function=eval_beamsize, function_kwargs={"generator": generator})
X = Xopt(generator=generator, evaluator=evaluator, vocs=vocs)
X.evaluator = Evaluator(function=eval_beamsize, function_kwargs={"generator": X.generator})
X.dump_file = run_dir + "nn_optimization_2d_1.yml"
X

In [ ]:
n_init = 3
initial_data_file = os.path.join(run_dir, f"optimization_2d_initial_data_n={n_init}.csv")

if os.path.isfile(initial_data_file):
    initial_data = pd.read_csv(initial_data_file)
    X.add_data(initial_data)
else:
    X.random_evaluate(n_init)
    X.data.to_csv(initial_data_file, index=False)

X.data

In [ ]:
%%time
for i in range(10):
    print(i)
    X.step()

In [ ]:
X.data.plot(y="total_size")

In [ ]:
X.data.plot(y=X.vocs.variable_names)

In [ ]:
X.generator.computation_time[["training", "acquisition_optimization"]].plot();

In [ ]:
# running minimum
running_min = []
for i in range(len(X.data)):
    running_min.append(X.data[vocs.objective_names[0]].iloc[:i+1].min())

fig, ax = plt.subplots(1, 1, figsize=(6, 4))
ax.plot(X.data.index.values, X.data[vocs.objective_names[0]].values, label="BO Sequence")
ax.plot(X.data.index.values, running_min, label="Running Minimum")
ax.set_ylabel(f"{vocs.objective_names[0]}")
ax.legend()
fig.tight_layout()

## Display GP model predictions

In [ ]:
X.data[["total_size" + k for k in ["", "_prior_mean", "_posterior_mean", "_posterior_sd"]]].plot();

In [ ]:
X.generator.visualize_model(show_prior_mean=True);